# Getting Started with AutoProf

In this notebook you will walk through the very basics of AutoProf functionality. Here you will learn how to make models; how to set them up for fitting; and how to view the results. These core elements will come up every time you use AutoProf, though in future notebooks you will learn how to take advantage of the advanced features in AutoProf.

In [ ]:
import os
import autoprof as ap
import numpy as np
import torch
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

## Your first model

The basic format for making an AutoProf model is given below. Once a model object is constructed, it can be manipulated and updated in various ways.

In [ ]:
model1 = ap.models.AutoProf_Model(
    name = "model1", # every model must have a unique name
    model_type = "sersic galaxy model", # this specifies the kind of model
    parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": 2, "Re": 10, "Ie": 1}, # here we set initial values for each parameter
)
model1.initialize() # before using the model it is good practice to call initialize so the model can get itself ready

# We can print the model's basic info
print(model1)

In [ ]:
# AutoProf has built in methods to plot relevant information. We didn't specify the region on the sky for
# this model to focus on, so by default it just makes a 100x100 window. Unless you are very lucky this wont
# line up with what you're trying to fit, so next we'll see how to give the model a target.

fig, ax = plt.subplots(figsize = (8,8))
ap.plots.model_image(fig, ax, model1)
plt.show()

## Giving the model a Target

Typically, the main goal when constructing an AutoProf model is to fit to an image. We need to give the model access to the image and some information about it to get started.

In [ ]:
# first let's download an image to play with
hdu = fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=36.3684&dec=-25.6389&size=700&layer=ls-dr9&pixscale=0.262&bands=r")
target_data = np.array(hdu[0].data, dtype = np.float64)

# Create a target object with specified pixelscale and zeropoint
target = ap.image.Target_Image(
    data = target_data,
    pixelscale = 0.262, # Every target image needs to know it's pixelscale in arcsec/pixel
    zeropoint = 22.5, # optionally you can give a zeropoint to tell AutoProf what the pixel flux units are
)

# The default AutoProf target plotting method uses log scaling in bright areas and histogram scaling in faint areas
fig3, ax3 = plt.subplots(figsize = (8,8))
ap.plots.target_image(fig3, ax3, target)
plt.show()

In [ ]:
# This model now has a target that it will attempt to match
model2 = ap.models.AutoProf_Model(
    name = "model with target", 
    model_type = "sersic galaxy model",
    target = target, # now the model knows what its trying to match
)

# Instead of giving initial values for all the parameters, it is possible to simply call "initialize" and AutoProf 
# will try to guess initial values for every parameter assuming the galaxy is roughly centered. It is also possible
# to set just a few parameters and let AutoProf try to figure out the rest. For example you could give it an initial
# Guess for the center and it will work from there.
model2.initialize()

# Plotting the initial parameters and residuals, we see it gets the rough shape of the galaxy right, but still has some fitting to do
fig4, ax4 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig4, ax4[0], model2)
ap.plots.residual_image(fig4, ax4[1], model2)
plt.show()

In [ ]:
# Now that the model has been set up with a target and initialized with parameter values, it is time to fit the image
result = ap.fit.LM(model2, method = 4, verbose = 1).fit()
# See that we use ap.fit.LM, this is the Levenberg-Marquardt Chi^2 minimization method, it is the recommended technique
# for most least-squares problems. However, there are situations in which different optimizers may be more desireable
# so the ap.fit package includes a few options to pick from. The various fitting methods will be described in a 
# different tutorial.
print(result.message) # the fitter will return a message about its convergence

In [ ]:
# we now plot the fitted model and the image residuals
fig5, ax5 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig5, ax5[0], model2)
ap.plots.residual_image(fig5, ax5[1], model2)
plt.show()

## Giving the model a specific target window

Sometimes an object isn't nicely centered in the image, and may not even be the dominant object in the image. It is therefore nice to be able to specify what part of the image we should analyze.

In [ ]:
model3 = ap.models.AutoProf_Model(
    name = "model with target", 
    model_type = "sersic galaxy model",
    target = target,
    window = [[480, 590],[555, 665]], # this is a region in pixel coordinates ((xmin,xmax),(ymin,ymax)) 
)

# We can plot the "model window" to show us what part of the image will be analyzed by that model
fig6, ax6 = plt.subplots(figsize = (8,8))
ap.plots.model_window(fig6, ax6, model3)
plt.show()

In [ ]:
model3.initialize()
result = ap.fit.LM(model3, method = 4, verbose = 1).fit()
print(result.message)

In [ ]:
# Note that when only a window is fit, the default plotting methods will only show that window
fig7, ax7 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig7, ax7[0], model3)
ap.plots.residual_image(fig7, ax7[1], model3)
plt.show()

## Setting parameter constraints

A common feature of fitting parameters is that they have some constraint on their behaviour and cannot be sampled at any value from (-inf, inf). AutoProf circumvents this by remapping any constrained parameter to a space where it can take any real value, at least for the sake of fitting. For most parameters these constraints are applied by default; for example the axis ratio q is required to be in the range (0,1). Other parameters, such as the position angle (PA) are cyclic, they can be in the range (0,pi) but also can wrap around. It is possible to manually set these constraints while constructing a model.

In general adding constraints makes fitting more difficult. There is a chance that the fitting process runs up against a constraint boundary and gets stuck. However, sometimes adding constraints is necessary and so the capability is included.

In [ ]:
# here we make a sersic model that can only have q and n in a narrow range
# Also, we give PA and initial value and lock that so it does not change during fitting
constrained_param_model = ap.models.AutoProf_Model(
    name = "constrained parameters", model_type = "sersic galaxy model", 
    parameters = {
        "q": {"limits": [0.4,0.6]}, 
        "n": {"limits": [2,3]}, 
        "PA": {"value": 60*np.pi/180, "locked": True},
    }
)

Aside from constraints on an individual parameter, it is sometimes desireable to have different models share parameter values. For example you may wish to combine multiple simple models into a more complex model (more on that in a different tutorial), and you may wish for them all to have the same center. This can be accomplished with "equality constraints" as shown below.

In [ ]:
# model 1 is a sersic model
model_1 = ap.models.AutoProf_Model(
    name = "constrained 1",
    model_type = "sersic galaxy model",
    parameters = {"center": [50,50], "PA": np.pi/4}
)
# model 2 is an exponential model
model_2 = ap.models.AutoProf_Model(
    name = "constrained 2",
    model_type = "exponential galaxy model",
)

# Here we add the constraint for "PA" to be the same for each model.
# In doing so we provide the parameter name and the model which should 
# be connected.
model_2.add_equality_constraint(model_1, "PA")

# Here we can see how the two models now both can modify this parameter
print("initial values: model_1 PA", model_1["PA"].value.item(), "model_2 PA", model_2["PA"].value.item())
# Now we modify the PA for model_1
model_1["PA"].value = np.pi/3
print("change model_1: model_1 PA", model_1["PA"].value.item(), "model_2 PA", model_2["PA"].value.item())
# Similarly we modify the PA for model_2
model_2["PA"].value = np.pi/2
print("change model_2: model_1 PA", model_1["PA"].value.item(), "model_2 PA", model_2["PA"].value.item())

In [ ]:
# Keep in mind that both models have full control over the parameter, it is listed in both of
# their "parameter_order" tuples. The built-in AutoProf functions keep track of constrained 
# parameters by asking models if any of their parameters are constrained
print("model_1 parameters: ", model_1.parameter_order(), " are any parameter constrained: ", model_1.equality_constraints)
print("model_2 parameters: ", model_2.parameter_order(), " are any parameter constrained: ", model_2.equality_constraints)

## PSF convolution

An important part of astronomical image analysis is accounting for PSF effects. To that end, AutoProf includes a number of approaches to handle PSF convolution. The main concept is that AutoProf will convolve its model with a PSF before comparing against an image. The PSF behaviour of a model is determined by the *psf_mode* parameter which can be set before fitting.

In [ ]:
# first a psf is needed, this is stored with the target object
# Here we simply construct a gaussian PSF image that is 31 pixels across
# Note the PSF must always be odd in its dimensions
xx, yy = np.meshgrid(np.linspace(-5,5,31), np.linspace(-5,5,31))
PSF = np.exp(-(xx**2 + yy**2)/5**2)
PSF /= np.sum(PSF)
target = ap.image.Target_Image(
    data = target_data,
    pixelscale = 0.262,
    zeropoint = 22.5,
    psf = PSF,
)

model_nopsf = ap.models.AutoProf_Model(
    name = "model without psf", 
    model_type = "sersic galaxy model",
    target = target,
    parameters = {"center": [90,90], "q": 0.6, "PA": 60*np.pi/180, "n": 2, "Re": 10, "Ie": 1},
    psf_mode = "none", # no PSF convolution will be done
)
model_psf = ap.models.AutoProf_Model(
    name = "model with psf", 
    model_type = "sersic galaxy model",
    target = target,
    parameters = {"center": [90,90], "q": 0.6, "PA": 60*np.pi/180, "n": 2, "Re": 10, "Ie": 1},
    psf_mode = "full", # now the full window will be PSF convolved
)
print("psf mode: ", model_psf.psf_mode)

# With a convolved sersic the center is much more smoothed out
fig, ax = plt.subplots(1,2,figsize = (16,7))
ap.plots.model_image(fig, ax[0], model_nopsf)
ax[0].set_title("No PSF convolution")
ap.plots.model_image(fig, ax[1], model_psf)
ax[1].set_title("With PSF convolution")
plt.show()

## Basic things to do with a model

Now that we know how to create a model and fit it to an image, lets get to know the model a bit better.

In [ ]:
# Save the model to a file

model2.save() # will default to save as AutoProf.yaml
with open("AutoProf.yaml", "r") as f:
    print(f.read()) # show what the saved file looks like

In [ ]:
# load a model from a file

# note that the target still must be specified, only the parameters are saved
model4 = ap.models.AutoProf_Model(name = "no name", filename = "AutoProf.yaml", target = target)
print(model4) # can see that it has been constructed with all the same parameters as the saved model2.

In [ ]:
# Save the model image to a file

model2.sample().save("model2.fits")

saved_image_hdu = fits.open("model2.fits")
fig, ax = plt.subplots(figsize = (8,8))
ax.imshow(
    np.log10(saved_image_hdu[0].data), 
    origin = "lower",
    cmap = ap.plots.visuals.cmap_grad, # gradient colourmap default for AutoProf
)
plt.show()

In [ ]:
# Save and load a target image

target.save("target.fits")

new_target = ap.image.Target_Image(filename = "target.fits")

fig, ax = plt.subplots(figsize = (8,8))
ap.plots.target_image(fig, ax, new_target)
plt.show()

In [ ]:
# Plot the surface brightness profile

fig8, ax8 = plt.subplots(figsize = (8,8))
ap.plots.galaxy_light_profile(fig8, ax8, model4)
plt.legend()
plt.show()

In [ ]:
# Give the model new parameter values manually

print("parameter input order: ", model4.parameter_order) # use this to see what order you have to give the parameters as input

# plot the old model
fig9, ax9 = plt.subplots(1,2,figsize = (16,7))
ap.plots.model_image(fig9, ax9[0], model4)
T = ax9[0].set_title("parameters as loaded")

# update and plot the new parameters
new_parameters = torch.tensor([75, 110, 0.4, 20*np.pi/180, 3, 25, 0.12]) # note that the center parameter needs two values as input
model4.initialize() # initialize must be called before optimization, or any other activity in which parameters are updated
model4.set_parameters(new_parameters) # full_sample will update the parameters, then run sample and return the model image 
ap.plots.model_image(fig9, ax9[1], model4)
T = ax9[1].set_title("new parameter values")

In [ ]:
# Access the model image pixels directly

fig2, ax2 = plt.subplots(figsize = (8,8))

pixels = model4.sample().data.detach().cpu().numpy()# model4.model_image.data is the pytorch stored model image pixel values. Calling detach().cpu().numpy() is needed to get the data out of pytorch and in a usable form

im = plt.imshow(
    np.log10(pixels), # take log10 for better dynamic range
    origin = "lower",
    cmap = ap.plots.visuals.cmap_grad, # gradient colourmap default for AutoProf
)
plt.colorbar(im)
plt.show()

In [ ]:
# Models can be constructed by providing model_type, or by creating the desired class directly

#                     notice this is no longer "AutoProf_Model"
model1_v2 = ap.models.Sersic_Galaxy(
    name = "model1 v2",
    parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": 2, "Re": 10, "Ie": 1},
)

# This will be the same as model1
print(model1_v2)

## Using GPU acceleration

This one is easy! If you have a cuda enabled GPU available, AutoProf will just automatically detect it and use that device. 

In [ ]:
# check if AutoProf has detected your GPU
print(ap.models.AutoProf_Model.device) # most likely this will say "cpu" unless you already have a cuda GPU, in which case it should say "cuda:0"

In [ ]:
# If you have a GPU but want to use the cpu for some reason, just set:
ap.models.AutoProf_Model.device = "cpu"
# before creating your models